In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time

# Initialize MediaPipe Hands and OpenCV
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize hand tracking
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Variables for controlling the cursor
prev_x, prev_y = None, None
last_click_time = time.time()

# Function to map hand position to screen coordinates
def map_to_screen(x, y, w, h):
    screen_width, screen_height = pyautogui.size()
    screen_x = screen_width * (x / w)
    screen_y = screen_height * (y / h)
    return int(screen_x), int(screen_y)

# Main loop for hand gesture recognition
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip the frame for mirror effect
    height, width, _ = frame.shape

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Hands
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Draw landmarks on the frame
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Get the tip of the index finger (for controlling the cursor)
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]

            # Map the hand's position (index finger tip) to the screen
            screen_x, screen_y = map_to_screen(index_tip.x, index_tip.y, 1, 1)

            # Move the cursor if position has changed
            if prev_x is not None and prev_y is not None:
                pyautogui.moveTo(screen_x, screen_y)

            prev_x, prev_y = screen_x, screen_y

            # Gesture: Pinch (Index + Middle fingers) to left-click
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

            # If index and middle fingers are close to each other
            if abs(index_tip.x - middle_tip.x) < 0.05 and abs(index_tip.y - middle_tip.y) < 0.05:
                if time.time() - last_click_time > 0.5:  # Debounce for click
                    pyautogui.click()
                    last_click_time = time.time()

    # Display the frame with landmarks
    cv2.imshow("Virtual Mouse", frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()